In [ ]:
import rdkit
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem as Chem

import subprocess #to run sh script via python
import optuna #is used to optimize hyperparameters
from joblib import Parallel, delayed #in parallel it is faster and order of rows is the same
import multiprocessing 

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, log_loss
from sklearn.ensemble import RandomForestClassifier

## Data preparation

##### Here we get rid of smiles, which cannot be processed by RDKit in case to avoid this issue in the future.

In [ ]:
path_of_file = input("Enter path of your database, which you want to use: ") #just path, without quotes
#"/home/gnss/dinakh/Git/QSAR_docking/Data_base_MOL_PORT/3_2023/Screening Compounds/merged.smi"
print(f"You chose '{path_of_file}' database")
radius= int(input("Enter the radius in Morgan fingerprint: "))
nBits= int(input("Enter the number of bits in Morgan fingerprint: "))
ecfp6_name = [f'Bit_{i}' for i in range(nBits)]
with open(path_of_file, "r") as ins:
    smiles = []
    for line in ins:
        smiles.append(line.split('\n')[0]) #convert a smiles format to the list, where which line of smile has its own row
print(f"Number compounds in your database '{len(smiles[1:])}'")

In [ ]:
number_of_compounds = input("Enter a number of compounds, which you want to sample on the first iteration: ") 
print(f"You chose '{number_of_compounds}' compounds")

In [ ]:
df_smiles_initial = pd.DataFrame(smiles[1:], columns=['smile'])
df_smiles_initial.head(3)

In [ ]:
list_right = []
bad_values = []
ECFP = [] #Extended Connectivity Fingerprint (ECFP)

for i in list(df_smiles_initial.index.values):
    try:
       list_right = Chem.MolFromSmiles((df_smiles_initial.iloc[i])['smile'])
       ECFP.append(Chem.GetMorganFingerprintAsBitVect(list_right,radius=radius, nBits=nBits))
    except: 
        bad_values.append(df_smiles_initial.iloc[i])
print(f"Number compounds did not processed by RDKit '{len(bad_values)}'")

bad_values_numeric = [] #for now it looks like a small dataframe for each compound and it is complex to reach the exact number of row 
for l in range(len(bad_values)): # so we will create a list of numbers from dataframe for convenience
    bad_values_numeric.append(bad_values[l].name) 
df_smiles_initial.drop(bad_values_numeric, inplace = True)
print(f"Final number of compounds in your database '{len(df_smiles_initial)}'")
df_smiles_initial.to_csv('df_smiles_initial.csv')

## Functions

In [ ]:
def from_sampled_to_glide_dock(x):
    smiles_to_list = x['smile'].tolist() #for docking it should be list of smiles
    smiles_to_list.insert(0, smiles[0]) #with the first row of names, which we take from the database
    with open('smiles_to_prepare.smi', 'w') as f:
        for line in smiles_to_list:
            f.write(f"{line}\n")

#### Confusion matrix

In [ ]:
def make_confusion_matrix(cf,    #fuction to create a nice cofusion matrix
                          group_names=None,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=True,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])

    if sum_stats:
        #Accuracy is sum of diagonal divided by total observations
        accuracy  = np.trace(cf) / float(np.sum(cf))

        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(
                accuracy,precision,recall,f1_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""

    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False
    
    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)

#### Prediction on the whole database

In [ ]:
def predict_values(x, model):
    molecula = Chem.MolFromSmiles(x)
    bit = Chem.GetMorganFingerprintAsBitVect(molecula,radius=radius, nBits=nBits)
    df_morgan_try  = pd.DataFrame(columns=ecfp6_name)
    df_morgan_try.loc[0] = list(bit)
    predictions1 = model.predict(df_morgan_try)[0]
    return predictions1

#### Function for working after docking

In [ ]:
def after_docking():
    df_dock_glide = pd.read_csv(r"glide-dock_ex.csv")
    print(f"Number compounds after docking '{len(df_dock_glide)}'")

    #ligprep generates different possible charges, absolute configuration and etc., therefore eventually we got table with duplicates in title and NO duplicates in SMILES themselves
    #however we will return to the initial database to exclude already processed compounds, so we need to keep it the same as it was 
    #we also need to get rid of compounds that were not docked because of reasons as incorrect configuration and etc. 

    print(f"Number duplicates '{df_dock_glide.title.duplicated().sum()}'")
    print(f"Number undocked compounds '{df_dock_glide['r_i_docking_score'].isna().sum()}'")
    df_dock_glide = df_dock_glide.dropna()

    df_dock_glide_final = pd.DataFrame(columns = df_dock_glide.columns)
    for x in df_dock_glide['title']:
        data_frame_small = df_dock_glide[df_dock_glide['title'] == x]
        min_glide_gscore = data_frame_small['r_i_docking_score'].min()
        data_frame_small_min_score = data_frame_small[data_frame_small['r_i_docking_score'] == min_glide_gscore]
        df_dock_glide_final= pd.concat([df_dock_glide_final, data_frame_small_min_score])
    df_dock_glide_final_dupl = df_dock_glide_final.drop_duplicates(subset='title', keep="last") #the final dataset has only one row for each title with the lowest docking score
    print(f"Number compounds in final dataset '{len(df_dock_glide_final_dupl)}'")
    #list_with_1_names = [] #here we just put the first part of smile to the dataframe with docking scores to merge them in a future
    return df_dock_glide_final_dupl#, list_with_1_names

In [ ]:
def return_full_names(x):
    data_frame_small = df_smiles_initial[df_smiles_initial['part_2'] == x]
    part_1_score = data_frame_small['part_1'].min()
    print(part_1_score)
    return part_1_score

#### Right form of the file after docking

In [ ]:
def preparation_of_right_form():
    global results_after_docking
    results_after_docking['part_1'] = part_1_lst
    list_of_full_names = []
    for row in results_after_docking.index:
        new_name = results_after_docking.loc[row, 'part_1'] + '\t' + results_after_docking.loc[row, 'title']
        list_of_full_names.append(new_name) #after docking, the program automatically take the first part as a smile and the second as a title, so 
    results_after_docking['full_title'] = list_of_full_names #we need to return the initial name to have the same prosedure for each iteration
    # 'full_title' is the column via we will exclude already processed compounds from the initial database
    results_after_docking = results_after_docking[['full_title', 'r_i_docking_score']] #we will work only with a docking scores ans smiles themselves, therefore keep only these columns

    ECFP = []
    for x in results_after_docking['full_title']:
        list_right = Chem.MolFromSmiles(x)
        ECFP.append(Chem.GetMorganFingerprintAsBitVect(list_right,radius=radius, nBits=nBits))
    ecfp6_bits = [list(l) for l in ECFP]
    results_after_docking[ecfp6_name] = ecfp6_bits #only presentaion of bits where each bit located in the separate column allow keep them from iteration to iteration
    results_after_docking.drop(results_after_docking[(results_after_docking['r_i_docking_score'] > 5)].index, inplace=True) 
    #sometimes Maestro can give us weird scores which we cannot take into account, the value 5 was chosed randomly, just based on the usual output that we get
    return results_after_docking

## FINAL LOOP

In [ ]:
labels = ['True Neg','False Pos','False Neg','True Pos']
categories = ['Zero', 'One']
number_of_iterations = 7
iteration = 1
cutoff = 0.1
cutoff_numeric_for_iteration_lst = []
MEAN_DOCK_SCORE_LST = []
LOWEST_SCORE_LST = []
while iteration < number_of_iterations:
  print(f"The iteration which you are working on: {iteration}")
  if iteration == 1:
    df_initial_compounds = df_smiles_initial.sample(n=int(number_of_compounds),replace=False) #defined by user in the beginning
    from_sampled_to_glide_dock(df_initial_compounds)


    file_ = open('shell_output.txt', 'w+') 
    subprocess.run('sh maestro_1_iteration.sh', shell=True, stdout=file_) 
    file_.close() 


    results_after_docking = after_docking()
    
    list_1_part = []
    list_2_part = []
    for i in df_smiles_initial['smile'].to_list():
      part_1 = i.split('\t', 1)[0]
      part_2 = i.split('\t', 1)[1]
      list_1_part.append(part_1)
      list_2_part.append(part_2)
    df_smiles_initial['part_1'] = list_1_part # we separate the titles in the initial database to the two part: smile that will be changed during the ligprep
    df_smiles_initial['part_2'] = list_2_part # (and it is the only stage in which we can pull out them) and the rest part 
    
    part_1_lst = Parallel(n_jobs=1)(delayed(return_full_names)(x) for x in results_after_docking['title']) #for now I cannot increase number of jobs, because it is run out of memory, but at least it can decrease the number of lines in the code
    df_docking_final_table = preparation_of_right_form()
    LOWEST_SCORE = df_docking_final_table['r_i_docking_score'].min()
    df_docking_final_table_initial = df_docking_final_table.copy()


    cutoff_numeric_for_iteration = df_docking_final_table['r_i_docking_score'].sort_values()[0:round(len(df_docking_final_table)*cutoff)].iloc[- 1]
    df_docking_final_table.loc[df_docking_final_table['r_i_docking_score'] >= cutoff_numeric_for_iteration, 'r_i_docking_score'] = 0 
    df_docking_final_table.loc[df_docking_final_table['r_i_docking_score'] < cutoff_numeric_for_iteration, 'r_i_docking_score'] = 1

    X_train, X_test, y_train, y_test = train_test_split(df_docking_final_table.drop(['r_i_docking_score', 'full_title'], axis=1),
                                                        df_docking_final_table['r_i_docking_score'], test_size=0.2,
                                                        random_state=42, stratify=df_docking_final_table['r_i_docking_score'])
    def objective(trial , X = X_train , y = y_train):
      train_x , valid_x , train_y , valid_y = train_test_split(X , y , \
              test_size = 0.2 , random_state = 42, stratify = y) 
      params = {
          'C' : trial.suggest_loguniform("C", 1e-2, 1)} #Inverse of regularization strength
      model1 = LogisticRegression(**params) # and so smaller values specify stronger regularization
      model1.fit(train_x , train_y)
      y_predlr = model1.predict(valid_x)
      ll = log_loss(valid_y , y_predlr)
      return ll
    
    # after the first iteration we should keep these sets (training and test) to augment them in the future
    df_TRAIN_INITIAL = df_docking_final_table_initial.loc[X_train.index.values.tolist()]
    df_TRAIN_INITIAL.to_csv('df_TRAIN_INITIAL.csv')
    MEAN_SCORE = df_TRAIN_INITIAL['r_i_docking_score'].mean()
    df_TEST_INITIAL = df_docking_final_table_initial.loc[X_test.index.values.tolist()]
    df_TEST_INITIAL.to_csv('df_TEST_INITIAL.csv')
    # in the first iteration we create these sets and after during the rest of itereatons we augmet them, so I suggest create a loop with if = 1 iteration : do creation, else do augmentation


    study = optuna.create_study()
    study.optimize(objective, n_trials=50) #number of attempts
    best_params = study.best_params
    found_x = best_params["C"]
    print(f"Inverse of regularization strength ('C'):  '{found_x}'") #the best parameter that was found and which will be used during training

    logmodel = LogisticRegression(C=round(found_x, 3), solver='sag') #we round the received number in case of avoidind strong adjustment to the particular dataset 
    logmodel.fit(X_train,y_train)
    predictions = logmodel.predict(X_test)
    cf_matrix = confusion_matrix(y_test,predictions)
    make_confusion_matrix(cf_matrix, 
                      group_names=labels,
                      categories=categories, 
                      cmap='Blues')
    

    datbase_no_train_test = df_smiles_initial[~df_smiles_initial['smile'].isin(df_docking_final_table['full_title'].to_list())]

    lst = Parallel(n_jobs=40)(delayed(predict_values)(x, model=logmodel) for x in datbase_no_train_test['smile']) #using parallel it takes 58min!
    datbase_no_train_test['docking_score'] = lst


    predicted_as_hits = datbase_no_train_test[datbase_no_train_test['docking_score'] == 1]
    if  len(predicted_as_hits) > 5000:
      predicted_as_hits = (predicted_as_hits[predicted_as_hits['docking_score'] == 1]).sample(n=5000,replace=False)


  else:
    from_sampled_to_glide_dock(predicted_as_hits)
    file_ = open('shell_output.txt', 'w+') 
    subprocess.run('sh maestro_delete.sh', shell=True, stdout=file_) 
    file_.close() 


    results_after_docking = after_docking()
    part_1_lst = Parallel(n_jobs=1)(delayed(return_full_names)(x) for x in results_after_docking['title']) #for now I cannot increase number of jobs, because it is run out of memory, but at least it can decrease the number of lines in the code
    df_for_merging = preparation_of_right_form()
    #LOWEST_SCORE = df_for_merging['r_i_docking_score'].min()
    train_merge = df_for_merging.sample(frac = 0.8)
    test_merge = df_for_merging.drop(train_merge.index)

    df_TRAIN_INITIAL = pd.read_csv(r"df_TRAIN_INITIAL.csv")
    df_TRAIN_INITIAL = df_TRAIN_INITIAL.drop(['Unnamed: 0'], axis=1)
    df_TRAIN_INITIAL = df_TRAIN_INITIAL.append(train_merge, ignore_index=True)
    MEAN_SCORE = df_TRAIN_INITIAL['r_i_docking_score'].mean()
    LOWEST_SCORE = df_TRAIN_INITIAL['r_i_docking_score'].min()
    df_TRAIN_INITIAL.to_csv('df_TRAIN_INITIAL.csv') #we need to save both sets in case to have numeric docking scores
    df_TEST_INITIAL = pd.read_csv(r"df_TEST_INITIAL.csv")
    df_TEST_INITIAL = df_TEST_INITIAL.drop(['Unnamed: 0'], axis=1)
    df_TEST_INITIAL = df_TEST_INITIAL.append(test_merge, ignore_index=True)
    df_TEST_INITIAL.to_csv('df_TEST_INITIAL.csv') #because after next actions they became as zeros and ones 
    #and I will also use it as output after docking protocol
    result_after_concat = pd.concat([df_TRAIN_INITIAL, df_TEST_INITIAL], axis=0)
    cutoff_numeric_for_iteration = df_TRAIN_INITIAL['r_i_docking_score'].sort_values()[0:round(len(df_TRAIN_INITIAL)*cutoff)].iloc[- 1]
    df_TRAIN_INITIAL.loc[df_TRAIN_INITIAL['r_i_docking_score'] >= cutoff_numeric_for_iteration, 'r_i_docking_score'] = 0 
    df_TRAIN_INITIAL.loc[df_TRAIN_INITIAL['r_i_docking_score'] < cutoff_numeric_for_iteration, 'r_i_docking_score'] = 1 
    df_TEST_INITIAL.loc[df_TEST_INITIAL['r_i_docking_score'] >= cutoff_numeric_for_iteration, 'r_i_docking_score'] = 0 
    df_TEST_INITIAL.loc[df_TEST_INITIAL['r_i_docking_score'] < cutoff_numeric_for_iteration, 'r_i_docking_score'] = 1
    X_train = df_TRAIN_INITIAL.drop(['r_i_docking_score', 'full_title'], axis=1)
    y_train = df_TRAIN_INITIAL['r_i_docking_score']
    X_test = df_TEST_INITIAL.drop(['r_i_docking_score', 'full_title'], axis=1)
    y_test = df_TEST_INITIAL['r_i_docking_score']

    def objective(trial , X = X_train , y = y_train):
      train_x , valid_x , train_y , valid_y = train_test_split(X , y , \
              test_size = 0.2 , random_state = 42, stratify = y) 
      params = {
          'C' : trial.suggest_loguniform("C", 1e-2, 1)} #Inverse of regularization strength
      model1 = LogisticRegression(**params) # and so smaller values specify stronger regularization
      model1.fit(train_x , train_y)
      y_predlr = model1.predict(valid_x)
      ll = log_loss(valid_y , y_predlr)
      return ll



    study = optuna.create_study()
    study.optimize(objective, n_trials=50) #number of attempts
    best_params = study.best_params
    found_x = best_params["C"]
    print(f"Inverse of regularization strength ('C'):  '{found_x}'") #the best parameter that was found and which will be used during training

    logmodel = LogisticRegression(C=round(found_x, 3), solver='sag') #we round the received number in case of avoidind strong adjustment to the particular dataset 
    logmodel.fit(X_train,y_train)
    predictions = logmodel.predict(X_test)
    cf_matrix = confusion_matrix(y_test,predictions)
    make_confusion_matrix(cf_matrix, 
                      group_names=labels,
                      categories=categories, 
                      cmap='Blues')

    datbase_no_train_test = df_smiles_initial[~df_smiles_initial['smile'].isin(result_after_concat['full_title'].to_list())]

    lst = Parallel(n_jobs=40)(delayed(predict_values)(x, model=logmodel) for x in datbase_no_train_test['smile']) #using parallel it takes 58min!
    datbase_no_train_test['docking_score'] = lst


    predicted_as_hits = datbase_no_train_test[datbase_no_train_test['docking_score'] == 1]
    if  len(predicted_as_hits) > 5000:
      predicted_as_hits = (predicted_as_hits[predicted_as_hits['docking_score'] == 1]).sample(n=5000,replace=False)
  cutoff -= 0.01
  cutoff_numeric_for_iteration_lst.append(cutoff_numeric_for_iteration)
  MEAN_DOCK_SCORE_LST.append(MEAN_SCORE)
  LOWEST_SCORE_LST.append(LOWEST_SCORE)
  iteration += 1
with open("cutoff_numeric_for_iteration_lst", "wb") as fp:
  pickle.dump(cutoff_numeric_for_iteration_lst, fp)
with open("MEAN_DOCK_SCORE_LST", "wb") as fp:
  pickle.dump(MEAN_DOCK_SCORE_LST, fp)
with open("LOWEST_SCORE_LST", "wb") as fp:
  pickle.dump(LOWEST_SCORE_LST, fp)